In [7]:
import pytest
from gwpy.signal import filter_design as gwpy_filter_functions
from gwpy.signal.window import planck as gwpy_planck
import torch
from ml4gw.utils.signal import fir_from_transfer, truncate_transfer, truncate_impulse, planck
import numpy
from scipy.special import expit
import torch
import matplotlib.pyplot as plt

In [12]:
def truncate_transfer(transfer, ncorner):
    out = transfer.clone()
    if out.ndim == 1:
        out = out[None, None, :]
    nsamp = out.size(-1)
    ncorner = ncorner if ncorner else 0
    out[:, :, 0:ncorner] = 0
    out[:, :, ncorner:nsamp] *= planck(nsamp-ncorner, nleft=5, nright=5)
    return out

In [13]:
def gwpy_truncate_transfer(transfer, ncorner):
    nsamp = transfer.size
    ncorner = ncorner if ncorner else 0
    out = transfer.copy()
    out[0:ncorner] = 0
    out[ncorner:nsamp] *= gwpy_planck(nsamp-ncorner, nleft=5, nright=5)
    return out

In [24]:
test = torch.randn((2, 2, 2048), dtype=torch.float64)

# calc with tensorized version
transfer = truncate_transfer(test, 1)

print(transfer[0][0], test[0][0] )
test = test.reshape(-1, test.shape[-1])
test_np = test.numpy().reshape(-1, test.shape[-1])
transfer = transfer.numpy().reshape(-1, test.shape[-1])


for tf, ts in zip(transfer, test_np):

    gwpy_tf = gwpy_truncate_transfer(ts, 1)
    print(tf[:10], gwpy_tf[:10], ts[:10])
    assert numpy.allclose(tf, gwpy_tf)

tensor([ 0.0000, -0.0000, -0.0076,  ...,  0.1152, -0.0452, -0.0000],
       dtype=torch.float64) tensor([-0.4504, -0.0241, -0.3289,  ...,  0.3804, -1.9672, -0.2148],
       dtype=torch.float64)
[ 0.         -0.         -0.00755637  0.40629046 -0.48294599 -0.10396961
 -1.31099483  0.58930341 -1.71372864  2.13713926] [ 0.         -0.         -0.00755637  0.40629045 -0.482946   -0.10396961
 -1.31099483  0.58930341 -1.71372864  2.13713926] [-0.45044439 -0.02412568 -0.32886135  1.34115497 -0.69283346 -0.10641474
 -1.31099483  0.58930341 -1.71372864  2.13713926]
[ 0.          0.         -0.02347598 -0.01824856  0.14787189  0.90895141
 -0.01652107  0.97560047  0.63175884  1.11458465] [ 0.          0.         -0.02347598 -0.01824856  0.14787189  0.90895139
 -0.01652107  0.97560047  0.63175884  1.11458465] [-3.14626452  0.45010345 -1.02170003 -0.06023806  0.21213675  0.93032788
 -0.01652107  0.97560047  0.63175884  1.11458465]
[ 0.         -0.          0.00365961 -0.87106768  0.69752528 -0.0887

In [17]:
test = torch.randn((2, 2, 2048), dtype=torch.float64)
test.numpy()
print(test)

tensor([[[ 0.4452, -0.4288, -1.3064,  ...,  0.3870,  0.2553, -0.1749],
         [-0.9742,  0.5201,  1.1972,  ..., -0.5162,  1.0109,  1.8718]],

        [[-1.3447,  1.1102, -0.1806,  ..., -1.2893,  0.5004,  2.3138],
         [ 1.6388, -1.1230, -0.0433,  ..., -0.2150, -0.9436, -1.5279]]],
       dtype=torch.float64)


In [33]:
max(planck(2048, nleft=5, nright=5) - gwpy_planck(2048, nleft=5, nright=5))

/Users/ethanmarx/miniconda3/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(1.7768e-08, dtype=torch.float64)

In [97]:
transfer_torch[0][1].numpy() == transfer_np

array([ True,  True,  True, ..., False, False,  True])

In [36]:
torch.max(1.7768e-08 * test)

tensor(6.7115e-08, dtype=torch.float64)

In [75]:

test = numpy.random.normal(size=16)

In [31]:
transfer = gwpy_truncate_transfer(test, 5)
ml4gw = truncate_transfer(torch.tensor(test), 5)
print(transfer, ml4gw)

AttributeError: 'Tensor' object has no attribute 'copy'

In [24]:
x = torch.arange(5, dtype=torch.float64) / 3.

In [48]:
torch.tensor(transfer).ndim

1

In [32]:
test = torch.randn((2, 2, 2048), dtype=torch.float64)

print(test)
transfer = truncate_transfer(test, 1)

print(test)

tensor([[[-0.8809, -0.2007,  1.1422,  ...,  1.8458, -0.3942, -0.6924],
         [-0.3670,  0.8146,  0.7530,  ..., -2.0083,  0.4756,  1.9663]],

        [[-0.9085, -0.5841, -0.7540,  ...,  0.2322,  2.2528, -1.9509],
         [-0.5257, -0.2869, -0.1822,  ...,  0.3735, -0.7505,  1.0400]]],
       dtype=torch.float64)
tensor([[[-0.8809, -0.2007,  1.1422,  ...,  1.8458, -0.3942, -0.6924],
         [-0.3670,  0.8146,  0.7530,  ..., -2.0083,  0.4756,  1.9663]],

        [[-0.9085, -0.5841, -0.7540,  ...,  0.2322,  2.2528, -1.9509],
         [-0.5257, -0.2869, -0.1822,  ...,  0.3735, -0.7505,  1.0400]]],
       dtype=torch.float64)


In [43]:
test = numpy.random.normal(size=1024)
def gwpy_truncate_transfer(transfer, ncorner):
    nsamp = transfer.size
    ncorner = ncorner if ncorner else 0
    out = transfer.copy()
    print(numpy.max(transfer - out))
    out[0:ncorner] = 0
    out[ncorner:nsamp] *= gwpy_planck(nsamp-ncorner, nleft=5, nright=5)

    transfer[0:ncorner] = 0
    transfer[ncorner:nsamp] *= gwpy_planck(nsamp-ncorner, nleft=5, nright=5)
    return out, transfer

In [44]:
out, transfer = gwpy_truncate_transfer(test, 1)

0.0


In [46]:
print(numpy.max(out - transfer))

0.0


In [47]:
test.dtype

dtype('float64')

In [38]:
test = torch.randn((1, 1, 1024))
def aug(t):
    clone = t.clone()
    clone[0, 0] = 0
    return clone

In [39]:
import ml4gw
print(test)
print(ml4gw.utils.signal.truncate_transfer(test, 0))
print(test)

tensor([[[ 0.5516,  1.2605, -1.7542,  ..., -0.7298,  0.3461,  0.2546]]])
tensor([[[ 0.0000,  0.0290, -0.5314,  ..., -0.2211,  0.0080,  0.0000]]])
tensor([[[ 0.0000,  0.0290, -0.5314,  ..., -0.2211,  0.0080,  0.0000]]])


/Users/ethanmarx/miniconda3/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
